# 演習
ディープラーニングを利用したSARSAを実装しましょう。Q学習のBrainクラス、エージェントクラスに変更を加えます。  
今回も同じく、強化学習により重力下で飛行する物体の制御を行います。


## 各設定
tensorflowとKerasのバージョンによっては、Kerasのコードでエラーが発生することがあります。エラーを回避するために、以下のセルでKerasのバージョンを指定してインストールします。  
以下のコードは、デフォルトのバージョンでエラーが発生しないときには必要ありません。

In [ ]:
!pip install keras==2.3  # 2020/3/28の時点ではtensorflow2.Xに対応するために必要

上記のセルの実行後、**ランタイム→ランタイムを再起動**によりバージョンの更新が完了します。  
念のために、以下のコードによりバージョンを確認しておきましょう。

In [ ]:
import tensorflow
import keras
print(tensorflow.__version__)
print(keras.__version__)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import animation, rc

from keras.models import Sequential
from keras.layers import Dense, ReLU
from keras.optimizers import RMSprop

optimizer = RMSprop()

### Brainクラス
エージェントの頭脳となるクラスです。Q値を出力するニューラルネットワークを構築し、Q値が正解に近づくように訓練します。  
SARSAに用いる式は以下の通りです。  

$$ Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \eta\left(R_{t+1}+\gamma Q(s_{t+1}, a_{t+1}) - Q(s_{t}, a_{t})\right) $$

ここで、$a_{t}$は行動、$s_t$は状態、$Q(s_t,a_t) $はQ値、$\eta$は学習係数、$R_{t+1}$は報酬、$\gamma$は割引率になります。  
Q学習では$t+1$の時刻における最大のQ値を使いますが、SARSAでは$t+1$において実際に選択した行動のQ値を使います。  
ディープラーニングの正解として用いるのは上記の式のうちの以下の部分です。  

$$R_{t+1}+\gamma Q(s_{t+1}, a_{t+1}) $$

以下の`Brain`クラスにおけるtrainメソッドでは、正解として上記を用います。  
まだコードが記載されてない箇所がありますので、上記の式を参考にコードを追記してSARSAを構築しましょう。




In [ ]:
class Brain:
    def __init__(self, n_state, n_mid, n_action, gamma=0.9, r=0.99):
        self.eps = 1.0  # ε
        self.gamma = gamma  # 割引率
        self.r = r  # εの減衰率

        model = Sequential()
        model.add(Dense(n_mid, input_shape=(n_state,)))
        model.add(ReLU()) 
        model.add(Dense(n_mid))
        model.add(ReLU()) 
        model.add(Dense(n_action))
        model.compile(loss="mse", optimizer=optimizer)
        self.model = model

    def train(self, states, next_states, action, next_action, reward, terminal):
        q = self.model.predict(states)  
        next_q = self.model.predict(next_states)
        t = np.copy(q)
        if terminal:
            t[:, action] = reward  #  エピソード終了時の正解は、報酬のみ
        else:
            t[:, action] =   # この行にコードを追記
        self.model.train_on_batch(states, t)

    def get_action(self, states):
        q = self.model.predict(states)
        if np.random.rand() < self.eps:
            action = np.random.randint(q.shape[1], size=q.shape[0])
        else:
            action = np.argmax(q, axis=1)
        if self.eps > 0.1:  # εの下限
            self.eps *= self.r
        return action

### エージェントのクラス
エージェントをクラスとして実装します。  
`step`メソッドにおいて、Q学習ではこの時刻における行動のみを使いましたが、SARSAではこの時刻における行動と次の時刻における行動を使います。

In [ ]:
class Agent:
    def __init__(self, v_x, v_y_sigma, v_jump, brain):
        self.v_x = v_x  # x速度
        self.v_y_sigma = v_y_sigma  # y速度、初期値の標準偏差
        self.v_jump = v_jump  # ジャンプ速度
        self.brain = brain
        self.reset()

    def reset(self):
        self.x = -1  # 初期x座標
        self.y = 0  # 初期y座標
        self.v_y = self.v_y_sigma * np.random.randn()  # 初期y速度
        states = np.array([[self.y, self.v_y]])
        self.action = self.brain.get_action(states)

    def step(self, g):  # 時間を1つ進める g:重力加速度
        states = np.array([[self.y, self.v_y]])
        self.x += self.v_x
        self.y += self.v_y

        reward = 0  # 報酬
        terminal = False  # 終了判定
        if self.x>1.0:
            reward = 1
            terminal = True
        elif self.y<-1.0 or self.y>1.0:
            reward = -1
            terminal = True
        reward = np.array([reward])

        if self.action[0] == 0:
            self.v_y -= g   # 自由落下
        else:
            self.v_y = self.v_jump  # ジャンプ
        next_states = np.array([[self.y, self.v_y]])

        next_action = self.brain.get_action(next_states)
        self.brain.train(states, next_states, self.action, next_action, reward, terminal)
        self.action = next_action

        if terminal:
            self.reset()

## 環境のクラス
環境をクラスとして実装します。このクラスの役割は、重力加速度を設定し、時間を前に進めるのみです。

In [ ]:
class Environment:
    def __init__(self, agent, g):
        self.agent = agent
        self.g = g

    def step(self):
        self.agent.step(self.g)
        return (self.agent.x, self.agent.y)

## アニメーション
今回は、matplotlibを使って物体の飛行をアニメーションで表します。  
アニメーションには、matplotlib.animationのFuncAnimation関数を使用します。  

In [ ]:
def animate(environment, interval, frames):
    fig, ax = plt.subplots()
    plt.close()
    ax.set_xlim(( -1, 1))
    ax.set_ylim((-1, 1))
    sc = ax.scatter([], [])

    def plot(data):
        x, y = environment.step()
        sc.set_offsets(np.array([[x, y]]))
        return (sc,)

    return animation.FuncAnimation(fig, plot, interval=interval, frames=frames, blit=True)

## SARSAの実行
εの減衰率`r`を0.99に設定し、SARSAを実行します。

In [ ]:
n_state = 2
n_mid = 32
n_action = 2
brain = Brain(n_state, n_mid, n_action, r=0.99)  # εが減衰する

v_x = 0.05
v_y_sigma = 0.1
v_jump = 0.2
agent = Agent(v_x, v_y_sigma, v_jump, brain)

g = 0.2
environment = Environment(agent, g)

anim = animate(environment, 50, 1024)
rc('animation', html='jshtml')
anim

エージェントが上下の端にぶつらずに飛べるようになることを確認しましょう。

# 解答例
以下は解答例です。  
どうしてもわからない時のみ、参考にしましょう。   



In [ ]:
class Brain:
    def __init__(self, n_state, n_mid, n_action, gamma=0.9, r=0.99):
        self.eps = 1.0  # ε
        self.gamma = gamma  # 割引率
        self.r = r  # εの減衰率

        model = Sequential()
        model.add(Dense(n_mid, input_shape=(n_state,)))
        model.add(ReLU()) 
        model.add(Dense(n_mid))
        model.add(ReLU()) 
        model.add(Dense(n_action))
        model.compile(loss="mse", optimizer=optimizer)
        self.model = model

    def train(self, states, next_states, action, next_action, reward, terminal):
        q = self.model.predict(states)  
        next_q = self.model.predict(next_states)
        t = np.copy(q)
        if terminal:
            t[:, action] = reward  #  エピソード終了時の正解は、報酬のみ
        else:
            t[:, action] = reward + self.gamma*next_q[:, next_action]  # この行にコードを追記
        self.model.train_on_batch(states, t)

    def get_action(self, states):
        q = self.model.predict(states)
        if np.random.rand() < self.eps:
            action = np.random.randint(q.shape[1], size=q.shape[0])
        else:
            action = np.argmax(q, axis=1)
        if self.eps > 0.1:  # εの下限
            self.eps *= self.r
        return action